## Viết nhanh level -1

In [32]:
import os
game_name = 'WelcomeToTheDungeon_v1'
start = ',\n  "-1": [\n   [],\n   [],\n   ['
print(start)
for i in os.listdir(f'A:\ENV\Agent\Ifelse\{game_name}'):
    print(f'    "{i[:-3]}"', end=',\n')
end = '   ]\n  ]'
print(end)

,
  "-1": [
   [],
   [],
   [
    "agentWelcomeToDungeon",
    "agentWTTD_v1",
   ]
  ]


In [ ]:
import json
a = json.load(open('A:\ENV\Log\level_game.json'))
for i in a:
    print(i, '-------------', list(a[i].keys())[2:])

## Kiểm tra hiệu suất của mô hình

In [1]:
## Kiểm tra hiện suất của hệ thống
# !git clone https://github.com/ngoxuanphong/ENV.git
%cd A:/ENV
import importlib.util
import warnings 
warnings.filterwarnings('ignore')
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning,NumbaExperimentalFeatureWarning, NumbaWarning
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaExperimentalFeatureWarning)
warnings.simplefilter('ignore', category=NumbaWarning)

from numba import njit
from numba.typed import List
import numba
import numpy as np
from setup import make
game_name = 'MachiKoro'
env = make(game_name)
getActionSize = env.getActionSize
getStateSize = env.getStateSize
getAgentSize = env.getAgentSize

getValidActions = env.getValidActions
getReward = env.getReward
numba_main_2 = env.numba_main_2

A:\ENV


In [2]:
from time import time
def test_game(game_name, dict_test = {}):
    env = make(game_name)
    getActionSize = env.getActionSize
    getValidActions = env.getValidActions
    getReward = env.getReward
    numba_main_2 = env.numba_main_2

    # @njit()
    def Test(state, perData):
        validActions = getValidActions(state)
        arr_action = np.where(validActions==1)[0]
        idx = np.random.randint(0, arr_action.shape[0])
        return arr_action[idx], perData
    
    per_test = [0]
    a = time()
    win1, per_test = numba_main_2(Test, 10000, per_test, 0)
    b = time()
    win2, per_test = numba_main_2(Test, 10000, per_test, 1)
    c = time()
    print(f'Game name: {game_name} | Win rate: ')
    print(f'        Level 0: {win1}/10000 in {int(b-a)} seconds')
    print(f'        Level 1: {win2}/10000 in {int(c-b)} seconds')
    try:
        win3, per_test = numba_main_2(Test, 10000, per_test, -1)
        d = time()
        time_lv_if = int(d-c)
    except:
        win3 = False
        time_lv_if = False
    if not win3:
        print(f'        Level -1: Chưa có level này')
    else:
        print(f'        Level -1: {win3}/10000 in {int(d-c)} seconds')
    dict_test[game_name] = {'Win rate': [win1, win2, win3], 
                            'Time': [int(b-a), int(c-b), time_lv_if]}
    return dict_test

test_game('Fantan')

Game name: Fantan | Win rate: 
        Level 0: 2506/10000 in 32 seconds
        Level 1: 97/10000 in 65 seconds
        Level -1: Chưa có level này


{'Fantan': {'Win rate': [2506, 97, False], 'Time': [32, 65, False]}}

In [3]:
import os
list_game = os.listdir('Base')
dict_test = {}
for game_name in ['Catan', 'CatanNoExchange', 'Century', 'Durak', 'Exploding_Kitten', 'Fantan', 'GoFish', 'Imploding_Kitten', 'MachiKoro', 'Phom', 'Poker', 'Sheriff', 'Splendor', 'Splendor_v2', 'Splendor_v3', 'StoneAge', 'SushiGo', 'TicketToRide', 'TLMN', 'WelcomeToTheDungeon_v1', 'WelcomeToTheDungeon_v2']:
    dict_test = test_game(game_name, dict_test)
import pandas as pd
df = pd.DataFrame(dict_test).T
df.to_csv('A:/ENV/Log/time_win.csv')

Game name: Catan | Win rate: 
        Level 0: 2535/10000 in 307 seconds
        Level 1: 211/10000 in 224 seconds
        Level -1: 13/10000 in 324 seconds
Game name: CatanNoExchange | Win rate: 
        Level 0: 2464/10000 in 190 seconds
        Level 1: 339/10000 in 673 seconds
        Level -1: Chưa có level này
Game name: Century | Win rate: 
        Level 0: 1932/10000 in 48 seconds
        Level 1: 12/10000 in 51 seconds
        Level -1: 1/10000 in 52 seconds
Game name: Durak | Win rate: 
        Level 0: 2561/10000 in 15 seconds
        Level 1: 447/10000 in 18 seconds
        Level -1: Chưa có level này
Game name: Exploding_Kitten | Win rate: 
        Level 0: 2002/10000 in 18 seconds
        Level 1: 1631/10000 in 16 seconds
        Level -1: Chưa có level này
Game name: Fantan | Win rate: 
        Level 0: 2413/10000 in 27 seconds
        Level 1: 117/10000 in 63 seconds
        Level -1: Chưa có level này
Game name: GoFish | Win rate: 
        Level 0: 2536/10000 in 10 sec

## Test lại agent level 2

In [ ]:
# Test lại agent lv2

def test_game(game_name):
    env = make(game_name)
    getActionSize = env.getActionSize
    getValidActions = env.getValidActions
    getReward = env.getReward
    numba_main_2 = env.numba_main_2

    @njit
    def Test(state,per):    
        actions = getValidActions(state)
        if per[-1][0][0][0][0] == 2:
            depth_tree = int(per[14][0][0][0][0])
            actions = getValidActions(state)
            output = per[-4][0][0][depth_tree] * actions + actions
            action = np.argmax(output)
        else:
            weight = per[-11][0][0][0]
            output = actions*weight + actions
            c = np.where(output == np.max(output))[0]
            action = np.random.choice(c)
            win = getReward(state)

            per[-11][0][0] += per[-9][0][0][action]
            if win != -1:
                per[-11][0][0] = np.zeros((1, getActionSize()))
        return action,per

    def convert_to_save(per):
        lst = []
        for i in range(len(per)):
            if len(per[i]) == 0:
                lst.append(List([np.zeros((1,1,1))]))
            else:
                lst.append(per[i])
        return lst

    def convert_to_test(lst):
        per = []
        for i in range(len(lst)):
            per.append(List(lst[i]))
        return per
    
    try:
        per_test = convert_to_test(np.load(f'A:\Trash\Data\{game_name}.npy', allow_pickle=True))
        win1, per_test = numba_main_2(Test, 10000, per_test, 1)
        win2, per_test = numba_main_2(Test, 10000, per_test, -1)
        print(f'Game name: {game_name} | Win rate: \n      Level 1: {win1}/10000 | Level -1: {win2}/10000')
    except:
        print(f'Game name: {game_name} | Game này chưa có level if else')

test_game('SushiGo')
for game_name in ['Splendor', 'TLMN', 'Catan']:
    test_game(game_name)
import os
list_game = os.listdir('A:\Trash\Data')
for game_name in list_game:
    test_game(game_name[:-4])